In [1]:
import re
import pandas as pd
import processing
from graphs import *

DATA_PATH = "./data/courses.json"

In [2]:
courses_df = pd.read_json(DATA_PATH, orient="index").filter(regex="math0.*", axis="index")

In [3]:
prereqs = courses_df["entry_pre_req"].apply(processing.prereq)

In [4]:
G = Graph()

course_to_vertex = {}
vertex_to_course = {}

for course, _ in prereqs.items():
    i = G.addVertex()
    course_to_vertex[course] = i
    vertex_to_course[i] = course

for course, prereq in prereqs.items():
    for pre_course in prereq:
        if pre_course in course_to_vertex.keys():
            G.addEdge(course_to_vertex[pre_course], course_to_vertex[course])



In [5]:
L = G.topSort()
course_order = list(reversed(list(map(lambda i : courses_df.loc[vertex_to_course[i]]["title"], L))))

In [6]:
courses_df.keys()

Index(['title', 'outcomes', 'reading', 'outline_school', 'outline_college',
       'outline_level_year', 'outline_availability', 'outline_scqf',
       'outline_ects', 'outline_summary', 'outline_description',
       'entry_pre_req', 'entry_co_req', 'entry_prohibited', 'entry_other_req',
       'entry_costs', 'visiting_pre_req', 'visiting_high_demand',
       'additional_skills', 'additional_keywords', 'contacts_organiser',
       'contacts_secretary', 'delivery_start', 'delivery_learning',
       'delivery_assessment', 'delivery_add_assessment', 'delivery_feedback',
       'delivery_skills', 'delivery_keywords', 'delivery_organiser',
       'delivery_secretary'],
      dtype='object')

In [7]:
scqf = courses_df["outline_level_year"]

SCQF_PATTERN = r"SCQF Level (?P<level>\d+) \((Year (?P<year>\d) (Undergraduate)|Postgraduate)\)"

def scqfGroup(level_year):
    m = re.match(SCQF_PATTERN, level_year)
    return m.groupdict()

courses_df["year"] = scqf.apply(lambda x : scqfGroup(x)["year"])
courses_df["level"] = scqf.apply(lambda x : scqfGroup(x)["level"])

In [8]:
courses_df

,title,outcomes,reading,outline_school,outline_college,outline_level_year,outline_availability,outline_scqf,outline_ects,outline_summary,...,delivery_learning,delivery_assessment,delivery_add_assessment,delivery_feedback,delivery_skills,delivery_keywords,delivery_organiser,delivery_secretary,year,level
math08073,Undergraduate Course: Mathematics for the Natu...,"On completion of this course, the student will...",Students will require a copy of the course tex...,School of Mathematics,College of Science and Engineering,SCQF Level 8 (Year 1 Undergraduate),Not available to visiting students,20,10.0,The course is a first university level course ...,...,"Total Hours:\n200\n(\nLecture Hours 22,\nSemin...","Written Exam\n80 %,\nCoursework\n20 %,\nPracti...","Coursework 20%, Examination: 80%.\nStudents re...",STACK questions (including practice) give feed...,Students will gain key skills in calculus appr...,"MNS1b,Sequences,series,power series,differenti...",Dr David Quinn\nTel:\nEmail: D.Quinn@ed.ac.uk,Mrs Frances Reid\nTel: (0131 6)50 4883\nEmail:...,1,8
math08074,Undergraduate Course: Engineering Mathematics ...,"On completion of this course, the student will...",Students will require a copy of the course tex...,School of Mathematics,College of Science and Engineering,SCQF Level 8 (Year 1 Undergraduate),Not available to visiting students,20,10.0,The course is a first university level course ...,...,"Total Hours:\n200\n(\nLecture Hours 22,\nSemin...","Written Exam\n80 %,\nCoursework\n20 %,\nPracti...","Coursework 20%, Examination: 80%.\nStudents re...",STACK questions (including practice) give feed...,Students will have key skills in basic algebra...,"EM1a,algebra,polynomials,functions,complex num...",Dr David Quinn\nTel:\nEmail: D.Quinn@ed.ac.uk,Mrs Frances Reid\nTel: (0131 6)50 4883\nEmail:...,1,8
math08075,Undergraduate Course: Engineering Mathematics ...,"On completion of this course, the student will...",Students will require a copy of the course tex...,School of Mathematics,College of Science and Engineering,SCQF Level 8 (Year 1 Undergraduate),Not available to visiting students,20,10.0,The course is a first university level course ...,...,"Total Hours:\n200\n(\nLecture Hours 22,\nSemin...","Written Exam\n80 %,\nCoursework\n20 %,\nPracti...","Coursework 20%, Examination: 80%.\nStudents re...",Auto-graded questions (including practice) giv...,Students will gain key skills in calculus appr...,"EM1b,Sequences,series,power series,differentia...",Dr David Quinn\nTel:\nEmail: D.Quinn@ed.ac.uk,Mrs Frances Reid\nTel: (0131 6)50 4883\nEmail:...,1,8
math07003,Undergraduate Course: Fundamentals of Algebra ...,"On completion of this course, the student will...",The main course material will be presented onl...,School of Mathematics,College of Science and Engineering,SCQF Level 7 (Year 1 Undergraduate),Available to all students,20,10.0,An introductory course in University Mathemati...,...,"Total Hours:\n200\n(\nLecture Hours 1,\nSemina...","Written Exam\n0 %,\nCoursework\n100 %,\nPracti...",80% calculated from scores on successful onlin...,Not entered,Not entered,FAC,Dr George Kinnear\nTel: (0131 6)50 5052\nEmail...,Mrs Frances Reid\nTel: (0131 6)50 4883\nEmail:...,1,7
math07004,Undergraduate Course: Introductory Mathematics...,"On completion of this course, the student will...",The main course material will be presented onl...,School of Mathematics,College of Science and Engineering,SCQF Level 7 (Year 1 Undergraduate),Available to all students,20,10.0,A foundational course in Mathematics focused o...,...,"Total Hours:\n200\n(\nLecture Hours 1,\nSemina...","Written Exam\n0 %,\nCoursework\n100 %,\nPracti...",Coursework : 100%\nExamination : 0%\nThere i...,Not entered,Not entered,IMA,Dr Robert Bickerton\nTel: (0131 6)50 5284\nEma...,Mrs Frances Reid\nTel: (0131 6)50 4883\nEmail:...,1,7
math08062,Undergraduate Course: Accelerated Algebra and ...,"On completion of this course, the student will...","David Poole, Linear Algebra; A modern introduc.

In [9]:
course_order

['Undergraduate Course: Proofs and Problem Solving (MATH08059)',
 'Undergraduate Course: Mathematics for the Natural Sciences 1a (MATH08072)',
 'Undergraduate Course: Introduction to Linear Algebra (MATH08057)',
 'Undergraduate Course: Introduction to Data Science (MATH08077)',
 'Undergraduate Course: Calculus and its Applications (MATH08058)',
 'Undergraduate Course: Facets of Mathematics (MATH08068)',
 'Undergraduate Course: Accelerated Proofs and Problem Solving (MATH08071)',
 'Undergraduate Course: Accelerated Algebra and Calculus for Direct Entry (MATH08062)',
 'Undergraduate Course: Several Variable Calculus and Differential Equations (MATH08063)',
 'Undergraduate Course: Probability (MATH08066)',
 'Undergraduate Course: Statistics (Year 2) (MATH08051)',
 'Undergraduate Course: Fundamentals of Pure Mathematics (MATH08064)',
 'Undergraduate Course: Computing and Numerics (MATH08065)',
 'Undergraduate Course: Introductory Mathematics with Applications (MATH07004)',
 'Undergraduate 